In [4]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical
f = open("big.txt")



In [3]:
nltk.download('punkt') #to import tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vineeth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
f = open("big.txt") # reading training data
data = f.read()
len(data)

3603366

In [12]:
 # replace all characters starting with \ to ' ' also convert everthing to lowercase
cleaned = re.sub(r'\W+', ' ',data[:100000]).lower()
#tokenize the cleaned data
tokens = word_tokenize(cleaned)
#input 3 words and checking 1 hence 3+1
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
print("text sequences")
print(text_sequences[:10])
# to eliminate all duplicate sequences
sequences = {}
count = 1       #
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
print("count")
print(count)
print("sequences ")
print(sequences[tokens[1]], tokens[1])
# converts the words to numbers to feed to the nural network
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 
print(sequences[:10])
#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1
n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]
print(np.shape(n_sequences),"\n\n\n",np.shape(sequences))

text sequences
[['the', 'project', 'gutenberg', 'ebook'], ['project', 'gutenberg', 'ebook', 'of'], ['gutenberg', 'ebook', 'of', 'the'], ['ebook', 'of', 'the', 'adventures'], ['of', 'the', 'adventures', 'of'], ['the', 'adventures', 'of', 'sherlock'], ['adventures', 'of', 'sherlock', 'holmes'], ['of', 'sherlock', 'holmes', 'by'], ['sherlock', 'holmes', 'by', 'sir'], ['holmes', 'by', 'sir', 'arthur']]
count
3140
sequences 
2 project
[[1, 488, 487, 634], [488, 487, 634, 5], [487, 634, 5, 1], [634, 5, 1, 859], [5, 1, 859, 5], [1, 859, 5, 103], [859, 5, 103, 26], [5, 103, 26, 49], [103, 26, 49, 163], [26, 49, 163, 1360]]
(18683, 4) 


 (18683, 4)


In [43]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
print(train_targets[0])


(18683, 3)

In [41]:
train_targets[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [44]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3, 3)              9417      
_________________________________________________________________
lstm_12 (LSTM)               (None, 3, 50)             10800     
_________________________________________________________________
lstm_13 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_12 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_13 (Dense)             (None, 3139)              160089    
Total params: 203,056
Trainable params: 203,056
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
584/584 [==============================] - 26s 22ms/step - loss: 6.9753 - accuracy: 0.0440
Epoch 

584/584 [==============================] - 16s 27ms/step - loss: 2.4359 - accuracy: 0.3998
Epoch 142/500
584/584 [==============================] - 15s 26ms/step - loss: 2.4333 - accuracy: 0.3997
Epoch 143/500
584/584 [==============================] - 15s 25ms/step - loss: 2.4320 - accuracy: 0.4006
Epoch 144/500
584/584 [==============================] - 15s 26ms/step - loss: 2.4141 - accuracy: 0.4065
Epoch 145/500
584/584 [==============================] - 15s 26ms/step - loss: 2.4313 - accuracy: 0.4036
Epoch 146/500
584/584 [==============================] - 15s 25ms/step - loss: 2.3815 - accuracy: 0.4079
Epoch 147/500
584/584 [==============================] - 15s 26ms/step - loss: 2.3707 - accuracy: 0.41410s - loss: 2.3700 - accuracy: 
Epoch 148/500
584/584 [==============================] - 14s 25ms/step - loss: 2.4050 - accuracy: 0.4050
Epoch 149/500
584/584 [==============================] - 15s 25ms/step - loss: 2.4061 - accuracy: 0.4075
Epoch 150/500
584/584 [================

584/584 [==============================] - 4s 6ms/step - loss: 1.7891 - accuracy: 0.5447
Epoch 296/500
584/584 [==============================] - 4s 6ms/step - loss: 1.8049 - accuracy: 0.5335
Epoch 297/500
584/584 [==============================] - 4s 6ms/step - loss: 1.8102 - accuracy: 0.5381
Epoch 298/500
584/584 [==============================] - 4s 6ms/step - loss: 1.8165 - accuracy: 0.5375
Epoch 299/500
584/584 [==============================] - 4s 6ms/step - loss: 1.8281 - accuracy: 0.5313
Epoch 300/500
584/584 [==============================] - 4s 6ms/step - loss: 1.7649 - accuracy: 0.5516
Epoch 301/500
584/584 [==============================] - 4s 6ms/step - loss: 1.7976 - accuracy: 0.5425
Epoch 302/500
584/584 [==============================] - 4s 6ms/step - loss: 1.7849 - accuracy: 0.5409
Epoch 303/500
584/584 [==============================] - 4s 6ms/step - loss: 1.8000 - accuracy: 0.5368
Epoch 304/500
584/584 [==============================] - 4s 6ms/step - loss: 1.7850 - a

584/584 [==============================] - 4s 7ms/step - loss: 1.4136 - accuracy: 0.6304
Epoch 454/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4535 - accuracy: 0.6160
Epoch 455/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4250 - accuracy: 0.6239
Epoch 456/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4470 - accuracy: 0.6186
Epoch 457/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4516 - accuracy: 0.6168
Epoch 458/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4549 - accuracy: 0.6162
Epoch 459/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4052 - accuracy: 0.6284
Epoch 460/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4053 - accuracy: 0.6332
Epoch 461/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4203 - accuracy: 0.6237
Epoch 462/500
584/584 [==============================] - 4s 7ms/step - loss: 1.4322 - a

In [22]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

no time to
[77, 142, 4] [[ 77 142   4]]
Next word suggestion: our
Next word suggestion: an
Next word suggestion: nothing


In [45]:
model.fit(train_inputs,train_targets,epochs=200,verbose=1)
model.save("mymodel1.h5")

Epoch 1/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3970 - accuracy: 0.6278
Epoch 2/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3929 - accuracy: 0.6314
Epoch 3/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3900 - accuracy: 0.6305
Epoch 4/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3759 - accuracy: 0.6328
Epoch 5/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3864 - accuracy: 0.6309
Epoch 6/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3917 - accuracy: 0.6292
Epoch 7/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3971 - accuracy: 0.6276
Epoch 8/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3750 - accuracy: 0.6309
Epoch 9/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3838 - accuracy: 0.6319
Epoch 10/200
584/584 [==============================] - 4s 7ms/step - loss: 1.3856 - accura

584/584 [==============================] - 15s 26ms/step - loss: 1.1701 - accuracy: 0.6801
Epoch 160/200
584/584 [==============================] - 15s 26ms/step - loss: 1.1668 - accuracy: 0.6830
Epoch 161/200
584/584 [==============================] - 16s 27ms/step - loss: 1.1604 - accuracy: 0.6863
Epoch 162/200
584/584 [==============================] - 15s 26ms/step - loss: 1.1533 - accuracy: 0.6844
Epoch 163/200
584/584 [==============================] - 15s 27ms/step - loss: 1.1303 - accuracy: 0.6902
Epoch 164/200
584/584 [==============================] - 15s 26ms/step - loss: 1.1355 - accuracy: 0.6885
Epoch 165/200
584/584 [==============================] - 17s 28ms/step - loss: 1.1596 - accuracy: 0.6830
Epoch 166/200
584/584 [==============================] - 16s 27ms/step - loss: 1.1455 - accuracy: 0.6905
Epoch 167/200
584/584 [==============================] - 16s 28ms/step - loss: 1.1648 - accuracy: 0.6827
Epoch 168/200
584/584 [==============================] - 15s 26ms/ste

In [46]:
print("hello")

hello


In [47]:
model.fit(train_inputs,train_targets,epochs=200,verbose=1)
model.save("mymodel2.h5")

Epoch 1/200
584/584 [==============================] - 15s 25ms/step - loss: 1.1057 - accuracy: 0.6988
Epoch 2/200
584/584 [==============================] - 15s 25ms/step - loss: 1.1227 - accuracy: 0.6915
Epoch 3/200
584/584 [==============================] - 14s 25ms/step - loss: 1.1112 - accuracy: 0.6961
Epoch 4/200
584/584 [==============================] - 15s 25ms/step - loss: 1.0885 - accuracy: 0.7038
Epoch 5/200
584/584 [==============================] - 14s 24ms/step - loss: 1.0881 - accuracy: 0.7012
Epoch 6/200
584/584 [==============================] - 14s 25ms/step - loss: 1.0897 - accuracy: 0.6995
Epoch 7/200
584/584 [==============================] - 14s 24ms/step - loss: 1.1051 - accuracy: 0.6988
Epoch 8/200
584/584 [==============================] - 14s 24ms/step - loss: 1.0864 - accuracy: 0.7000
Epoch 9/200
584/584 [==============================] - 15s 25ms/step - loss: 1.0763 - accuracy: 0.7044
Epoch 10/200
584/584 [==============================] - 14s 25ms/step - l

584/584 [==============================] - 4s 7ms/step - loss: 0.9630 - accuracy: 0.7352
Epoch 155/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9347 - accuracy: 0.7392
Epoch 156/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9372 - accuracy: 0.7362
Epoch 157/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9398 - accuracy: 0.7384
Epoch 158/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9704 - accuracy: 0.7292
Epoch 159/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9350 - accuracy: 0.7378
Epoch 160/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9449 - accuracy: 0.7368
Epoch 161/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9176 - accuracy: 0.7415
Epoch 162/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9277 - accuracy: 0.7417
Epoch 163/200
584/584 [==============================] - 4s 7ms/step - loss: 0.9307 - a

In [48]:
model.fit(train_inputs,train_targets,epochs=100,verbose=1)
model.save("mymodel3.h5")

Epoch 1/100
584/584 [==============================] - 4s 7ms/step - loss: 0.9004 - accuracy: 0.7488
Epoch 2/100
584/584 [==============================] - 4s 6ms/step - loss: 0.9184 - accuracy: 0.7432
Epoch 3/100
584/584 [==============================] - 4s 6ms/step - loss: 0.9189 - accuracy: 0.7389
Epoch 4/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8868 - accuracy: 0.7500
Epoch 5/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8702 - accuracy: 0.7558
Epoch 6/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8990 - accuracy: 0.7481
Epoch 7/100
584/584 [==============================] - 4s 7ms/step - loss: 0.9038 - accuracy: 0.7443
Epoch 8/100
584/584 [==============================] - 4s 7ms/step - loss: 0.9428 - accuracy: 0.7368
Epoch 9/100
584/584 [==============================] - 4s 7ms/step - loss: 0.9059 - accuracy: 0.7478
Epoch 10/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8924 - accura

584/584 [==============================] - 4s 6ms/step - loss: 0.8260 - accuracy: 0.7652
Epoch 82/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8759 - accuracy: 0.7512
Epoch 83/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8718 - accuracy: 0.7526
Epoch 84/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8127 - accuracy: 0.7694
Epoch 85/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8040 - accuracy: 0.7695
Epoch 86/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8233 - accuracy: 0.7652
Epoch 87/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8725 - accuracy: 0.7514
Epoch 88/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8278 - accuracy: 0.7649
Epoch 89/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8165 - accuracy: 0.7674
Epoch 90/100
584/584 [==============================] - 4s 6ms/step - loss: 0.8384 - accuracy: 

In [49]:
model.fit(train_inputs,train_targets,epochs=100,verbose=1)
model.save("mymodel3.h5")

Epoch 1/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8373 - accuracy: 0.7635
Epoch 2/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8168 - accuracy: 0.7700
Epoch 3/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8501 - accuracy: 0.7587
Epoch 4/100
584/584 [==============================] - 4s 7ms/step - loss: 0.8287 - accuracy: 0.7649
Epoch 5/100
584/584 [==============================] - 12s 20ms/step - loss: 0.8016 - accuracy: 0.7713
Epoch 6/100
584/584 [==============================] - 17s 29ms/step - loss: 0.7913 - accuracy: 0.7748
Epoch 7/100
584/584 [==============================] - 17s 30ms/step - loss: 0.8153 - accuracy: 0.7709
Epoch 8/100
584/584 [==============================] - 18s 31ms/step - loss: 0.8721 - accuracy: 0.7515
Epoch 9/100
584/584 [==============================] - 16s 28ms/step - loss: 0.8573 - accuracy: 0.7560
Epoch 10/100
584/584 [==============================] - 16s 28ms/step - loss: 0.8

In [50]:
model.fit(train_inputs,train_targets,epochs=100,verbose=1)
model.save("mymodel3.h5")

Epoch 1/100
584/584 [==============================] - 17s 28ms/step - loss: 0.7699 - accuracy: 0.7787
Epoch 2/100
584/584 [==============================] - 16s 28ms/step - loss: 0.7592 - accuracy: 0.7826
Epoch 3/100
584/584 [==============================] - 16s 27ms/step - loss: 0.7438 - accuracy: 0.7884
Epoch 4/100
584/584 [==============================] - 16s 28ms/step - loss: 0.7594 - accuracy: 0.7810
Epoch 5/100
584/584 [==============================] - 16s 28ms/step - loss: 0.7679 - accuracy: 0.7804
Epoch 6/100
584/584 [==============================] - 16s 28ms/step - loss: 0.7761 - accuracy: 0.7794
Epoch 7/100
584/584 [==============================] - 16s 28ms/step - loss: 0.7697 - accuracy: 0.7792
Epoch 8/100
584/584 [==============================] - 16s 28ms/step - loss: 0.7805 - accuracy: 0.7762
Epoch 9/100
584/584 [==============================] - 17s 29ms/step - loss: 0.7804 - accuracy: 0.77640s
Epoch 10/100
584/584 [==============================] - 17s 28ms/step -

In [51]:
model.fit(train_inputs,train_targets,epochs=100,verbose=1)
model.save("mymodel4.h5")

Epoch 1/100
584/584 [==============================] - 15s 25ms/step - loss: 0.7011 - accuracy: 0.7943
Epoch 2/100
584/584 [==============================] - 15s 25ms/step - loss: 0.6878 - accuracy: 0.8015
Epoch 3/100
584/584 [==============================] - 14s 25ms/step - loss: 0.7164 - accuracy: 0.7926
Epoch 4/100
584/584 [==============================] - 14s 25ms/step - loss: 0.7083 - accuracy: 0.7927
Epoch 5/100
584/584 [==============================] - 14s 25ms/step - loss: 0.6963 - accuracy: 0.7965
Epoch 6/100
584/584 [==============================] - 15s 25ms/step - loss: 0.7627 - accuracy: 0.7801
Epoch 7/100
584/584 [==============================] - 14s 25ms/step - loss: 0.7315 - accuracy: 0.7878
Epoch 8/100
584/584 [==============================] - 15s 25ms/step - loss: 0.6726 - accuracy: 0.8071
Epoch 9/100
584/584 [==============================] - 15s 25ms/step - loss: 0.7041 - accuracy: 0.7954
Epoch 10/100
584/584 [==============================] - 15s 25ms/step - l

In [52]:
model.fit(train_inputs,train_targets,epochs=100,verbose=1)
model.save("mymodel5.h5")

Epoch 1/100
584/584 [==============================] - 16s 28ms/step - loss: 0.6529 - accuracy: 0.8098
Epoch 2/100
584/584 [==============================] - 14s 23ms/step - loss: 0.6657 - accuracy: 0.80355s - loss: 0.6324 - 
Epoch 3/100
584/584 [==============================] - 4s 7ms/step - loss: 0.6471 - accuracy: 0.8062
Epoch 4/100
584/584 [==============================] - 4s 7ms/step - loss: 0.6359 - accuracy: 0.8137
Epoch 5/100
584/584 [==============================] - 4s 7ms/step - loss: 0.6437 - accuracy: 0.8091
Epoch 6/100
584/584 [==============================] - 4s 7ms/step - loss: 0.7108 - accuracy: 0.7937
Epoch 7/100
584/584 [==============================] - 4s 7ms/step - loss: 0.6636 - accuracy: 0.8051
Epoch 8/100
584/584 [==============================] - 4s 7ms/step - loss: 0.6199 - accuracy: 0.8182
Epoch 9/100
584/584 [==============================] - 4s 7ms/step - loss: 0.6409 - accuracy: 0.8121
Epoch 10/100
584/584 [==============================] - 4s 7ms/step